## Reference

#### Giba
- https://www.kaggle.com/titericz/the-property-by-giba

#### Jiazhen Xi 
- https://www.kaggle.com/johnfarrell/giba-s-property-extended-result
- https://www.kaggle.com/johnfarrell/baseline-with-lag-select-fake-rows-dropped

#### Mohsin hasan 
- https://www.kaggle.com/tezdhar/breaking-lb-fresh-start

####  Dmitry Frumkin 
- https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2

#### Paradox
- https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/61472#363394

In [1]:
%matplotlib inline
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import matplotlib.pyplot as plt

## Loading Data

In [3]:
train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

In [41]:
transact_cols = [f for f in train_df.columns if f not in ["ID", "target"]]
y = np.log1p(train_df["target"]).values

test_df["target"] = train_df["target"].mean()

## Leak Features

In [15]:
#Refence Jiazhen Xi
rows = [1757, 3809, 511, 3798, 625, 3303, 4095, 1283, 4209, 1696, 3511, 816, 245, 
        1383, 2071, 3492, 378, 2971,2366, 4414, 2790, 3979, 193, 1189, 3516, 810, 
        4443, 3697, 235, 1382, 4384, 3418, 4396, 921, 3176, 650]

In [36]:
#Refence Giba
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', 
         '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 
         'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', 
         '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992', 
         'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', 
         '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
         '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2', '0572565c2', 
         '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98' ]

In [37]:
train.loc[rows, ['target'] + cols]

,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,...,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
1757,115636.36,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3809,834800.00,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
511,296444.44,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3798,247166.66,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
625,550000.00,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3303,3076666.66,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4095,440000.00,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1283,1600000.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4209,466461.54,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1696,3147200.00,1600000.00,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76


## Extend Leak Features

In [38]:
train_silce = train.loc[rows, :].drop('ID', axis=1)

####  fork from: https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2

In [ ]:
def _get_leak(df, cols, lag=0):
    d1 = df[cols[:-lag-2]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = df[cols[lag+2:]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = df[cols[lag]]
    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    return d1.merge(d3, how='left', on='key').pred.fillna(0)
    

#### fork from: https://www.kaggle.com/tezdhar/breaking-lb-fresh-start

In [42]:
def compiled_leak_result():
    
    max_nlags = len(cols) - 2
    train_leak = train[["ID", "target"] + cols]
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        train_leak[c] = _get_leak(train_leak, cols, i)
        
        leaky_cols.append(c)
        train_leak = train.join(
            train_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in train", leaky_value_counts[-1])
        print(
            "% of correct leaks values in train ", 
            leaky_value_corrects[-1]
        )
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        print(
            'Score (filled with nonzero mean)', 
            scores[-1]
        )
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

In [ ]:
train_leak, result = compiled_leak_result()

In [ ]:
result = pd.DataFrame.from_dict(result, orient='columns')
result.T

In [ ]:
result.to_csv('train_leaky_stat.csv', index=False)

In [ ]:
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

In [ ]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [ ]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]
train_leak = rewrite_compiled_leak(train_leak, best_lag)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

In [ ]:
train_res = train_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak.csv', index=False)

## result

In [ ]:
def compiled_leak_result_test(max_nlags):
    test_leak = test[["ID", "target"] + cols]
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        test_leak[c] = _get_leak(test_leak, cols, i)
        
        leaky_cols.append(c)
        test_leak = test.join(
            test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        print("Leak values found in test", leaky_value_counts[-1])
        
    result = dict(leaky_count=leaky_value_counts,)
    
    return test_leak, result

In [ ]:
test_leak, test_result = compiled_leak_result_test(max_nlags=38)

In [ ]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

In [ ]:
test_result.to_csv('test_leaky_stat.csv', index=False)

In [ ]:
test_leak = rewrite_compiled_leak(test_leak, best_lag)
test_leak[['ID']+leaky_cols+['compiled_leak']].head()

In [ ]:
test_res = test_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak.csv', index=False)

In [ ]:
test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]

## submission

In [ ]:
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)

In [ ]:
dnn_lgb_result = pd.read_csv('submit.csv')
finally_sub = sub.copy()
finally_sub[sub['target'] == 0] = dnn_lgb_result[sub['target'] == 0]
finally_sub.to_csv(f"submission2.csv", index=False)